# Setting up an OpenCMISS-Iron problem from scratch - jupyter notebook

This is an example script to solve a Laplace problem using OpenCMISS-Iron calls
 in python ike all python examples with importing modules.

In mathematics and physics, Laplace's equation is a second-order partial differential equation
named after Pierre-Simon Laplace who first studied its properties.

## Loading OpenCMISS-Iron library
In order to use OpenCMISS we have to first import the opencmiss.iron module
from the opencmiss package. This initialises library with default values and
seeds random generator with a new value

In [1]:
# Intialise OpenCMISS-Iron.
from opencmiss.iron import iron
print('Hello world')

Hello world


Assuming OpenCMISS has been correctly built with the Python bindings by
following the instructions in the programmer documentation, we can now access
all the OpenCMISS functions, classes and constants under the iron namespace.

The next section describes how we can interact with the OpenCMISS-Iron library
through an object-oriented API.

In [2]:
# Set problem parameters
height = 1.0
width = 2.0
length = 3.0

#We need constant identificators for various 
#library functional points
(coordinateSystemUserNumber,
    regionUserNumber,
    basisUserNumber,
    generatedMeshUserNumber,
    meshUserNumber,
    decompositionUserNumber,
    geometricFieldUserNumber,
    equationsSetFieldUserNumber,
    dependentFieldUserNumber,
    equationsSetUserNumber,
    problemUserNumber) = range(1,12)

In [3]:
#initialise parameters of the model
numberGlobalXElements = 5
numberGlobalYElements = 5
numberGlobalZElements = 5

Get the computational nodes information. Note that in order for the model to
utillise parallel computations the code must be running under the supervision
of an MPI library and as such must be executed with mpirun/mpiexec assistance
If mpirun/mpiexec is not used, number of nodes witll be 1 and only the master
node will be created.

In [4]:
numberOfComputationalNodes = iron.ComputationalNumberOfNodesGet()
computationalNodeNumber = iron.ComputationalNodeNumberGet()

In [5]:

# The first step in our example will be to initialise OpenCMISS an to set up a region 
# and coordinate system for our 3D mesh.

# Creation a RC coordinate system
coordinateSystem = iron.CoordinateSystem()
coordinateSystem.CreateStart(coordinateSystemUserNumber)
coordinateSystem.dimension = 3
coordinateSystem.CreateFinish()

# Create a region
region = iron.Region()
region.CreateStart(regionUserNumber,iron.WorldRegion)
region.label = "LaplaceRegion"
region.coordinateSystem = coordinateSystem
region.CreateFinish()

In [6]:
# In this example we will use the generated mesh capabilities of OpenCMISS to 
# generate our 3D mesh. The first thing we need to do is create a basis function 
# for the mesh. We will define a trilinear Lagrange basis.
# Create a tri-linear lagrange basis
basis = iron.Basis()
basis.CreateStart(basisUserNumber)
basis.type = iron.BasisTypes.LAGRANGE_HERMITE_TP
basis.numberOfXi = 3
basis.interpolationXi = [iron.BasisInterpolationSpecifications.LINEAR_LAGRANGE]*3
basis.quadratureNumberOfGaussXi = [2]*3
basis.CreateFinish()

In [7]:
# We can now create a generated mesh. We will create a regular mesh of size 
# width x height x length and divide the mesh into numberOfXElements in the X direction,
# numberOfYElements in the Y direction and numberOfZElements in the Z direction. 
# Note that when we finish generating the mesh we have a mesh object returned to us. 
# This mesh object is just the same as if we had manually created the regular mesh.

#  Create a generated mesh
generatedMesh = iron.GeneratedMesh()
generatedMesh.CreateStart(generatedMeshUserNumber,region)
generatedMesh.type = iron.GeneratedMeshTypes.REGULAR
generatedMesh.basis = [basis]
generatedMesh.extent = [width,height,length]
generatedMesh.numberOfElements = [numberGlobalXElements,numberGlobalYElements,numberGlobalZElements]

In [8]:
# Let's double check the number of computational nodes we use
print(numberOfComputationalNodes)

1


In [9]:
mesh = iron.Mesh()
generatedMesh.CreateFinish(meshUserNumber,mesh)


In [10]:
# Once the mesh has been created we can decompose it into a number of domains in order to allow for 
# parallelism. We choose the options to let OpenCMISS calculate the best way to break up the mesh. 
# We also set the number of domains to be equal to the number of computational nodes this example is running on.
# Note that if MPI infrastructure is not used, only single domain will ve created

# Create a decomposition for the mesh
decomposition = iron.Decomposition()
decomposition.CreateStart(decompositionUserNumber,mesh)
decomposition.type = iron.DecompositionTypes.CALCULATED
decomposition.numberOfDomains = numberOfComputationalNodes
decomposition.CreateFinish()

In [11]:
# Now that the mesh has been decomposed we are in a position to create fields. The first field we need to 
# create is the geometry field. Here we create a field and set the field’s mesh decomposition to the 
# decomposed mesh that we have just created. We can choose exact how each component of the field is 
# interpolated by setting component mesh component to be the mesh components that we created for the mesh. 
# For this example we only have one mesh component. Once we have finished creating the field we can change 
# the field DOFs to give us our geometry. Since this mesh has been generated we can use the generated mesh 
# object to calculate the geometric parameters of the regular mesh.

# Create a field for the geometry
geometricField = iron.Field()
geometricField.CreateStart(geometricFieldUserNumber,region)
geometricField.meshDecomposition = decomposition
geometricField.ComponentMeshComponentSet(iron.FieldVariableTypes.U,1,1)
geometricField.ComponentMeshComponentSet(iron.FieldVariableTypes.U,2,1)
geometricField.ComponentMeshComponentSet(iron.FieldVariableTypes.U,3,1)
geometricField.CreateFinish()

In [12]:
# Set geometry from the generated mesh
generatedMesh.GeometricParametersCalculate(geometricField)

In [13]:
# We are now in a position to define the type of physics that we wish to solve. This is done by creating an 
# equations set which is a contianer object for all the parameters we need to describe the physics. 
# Here we create a standard Laplace equations set.

# {\displaystyle \nabla ^{2}f={\frac {\partial ^{2}f}{\partial x^{2}}}+{\frac {\partial ^{2}f}{\partial y^{2}}}+{\frac {\partial ^{2}f}{\partial z^{2}}}=0.}

# Create standard Laplace equations set
equationsSetField = iron.Field()
equationsSet = iron.EquationsSet()
equationsSetSpecification = [iron.EquationsSetClasses.CLASSICAL_FIELD,
        iron.EquationsSetTypes.LAPLACE_EQUATION,
        iron.EquationsSetSubtypes.STANDARD_LAPLACE]
equationsSet.CreateStart(equationsSetUserNumber,region,geometricField,
        equationsSetSpecification,equationsSetFieldUserNumber,equationsSetField)
equationsSet.CreateFinish()

In [14]:
# For the Laplace equation we need a dependent field (our solution).
# Here we do not define a field before the create starts and so we let OpenCMISS create an appropriate 
# dependent field for the Laplace equations being described. Once the fields have been created we can
# set the field DOF values. 

# Create dependent field
dependentField = iron.Field()
equationsSet.DependentCreateStart(dependentFieldUserNumber,dependentField)
dependentField.DOFOrderTypeSet(iron.FieldVariableTypes.U,iron.FieldDOFOrderTypes.SEPARATED)
dependentField.DOFOrderTypeSet(iron.FieldVariableTypes.DELUDELN,iron.FieldDOFOrderTypes.SEPARATED)
equationsSet.DependentCreateFinish()

# Initialise dependent field
dependentField.ComponentValuesInitialiseDP(iron.FieldVariableTypes.U,iron.FieldParameterSetTypes.VALUES,1,0.5)


In [15]:
# Create equations 
equations = iron.Equations()
equationsSet.EquationsCreateStart(equations)
equations.sparsityType = iron.EquationsSparsityTypes.SPARSE
equations.outputType = iron.EquationsOutputTypes.NONE
equationsSet.EquationsCreateFinish()


In [16]:
# Now we are ready to set up a problem to be solved by OpenCMISS
# Create Laplace problem

problem = iron.Problem()
problemSpecification = [iron.ProblemClasses.CLASSICAL_FIELD,
        iron.ProblemTypes.LAPLACE_EQUATION,
        iron.ProblemSubtypes.STANDARD_LAPLACE]
problem.CreateStart(problemUserNumber, problemSpecification)
problem.CreateFinish()

In [17]:
# OpenCMISS control loop is a "supervisor" for the computational process.
# Create control loops
problem.ControlLoopCreateStart()
problem.ControlLoopCreateFinish()

In [18]:
# Now we are ready to setup a solver for the problem. We need to specify solver type (iterative) and tolerances 
# in order for the computational loop to maintain control over the solution

# Create problem solver
solver = iron.Solver()
problem.SolversCreateStart()
problem.SolverGet([iron.ControlLoopIdentifiers.NODE],1,solver)
solver.outputType = iron.SolverOutputTypes.SOLVER
solver.linearType = iron.LinearSolverTypes.ITERATIVE
solver.linearIterativeAbsoluteTolerance = 1.0E-12
solver.linearIterativeRelativeTolerance = 1.0E-12
problem.SolversCreateFinish()


In [19]:
# Create solver equations and add equations set to solver equations
solver = iron.Solver()
solverEquations = iron.SolverEquations()
problem.SolverEquationsCreateStart()
problem.SolverGet([iron.ControlLoopIdentifiers.NODE],1,solver)
solver.SolverEquationsGet(solverEquations)
solverEquations.sparsityType = iron.SolverEquationsSparsityTypes.SPARSE
equationsSetIndex = solverEquations.EquationsSetAdd(equationsSet)
problem.SolverEquationsCreateFinish()

In [20]:
# The Dirichlet problem for Laplace's equation consists of finding a solution φ on some domain D such that φ on 
# the boundary of D is equal to some given function. Since the Laplace operator appears in the heat equation, 
# one physical interpretation of this problem is as follows: fix the temperature on the boundary of the domain 
# according to the given specification of the boundary condition. Allow heat to flow until a stationary state is 
# reached in which the temperature at each point on the domain doesn't change anymore. 
# The temperature distribution in the interior will then be given by the solution to the corresponding 
# Dirichlet problem.

# Create boundary conditions and set first and last nodes to 0.0 and 1.0
boundaryConditions = iron.BoundaryConditions()
solverEquations.BoundaryConditionsCreateStart(boundaryConditions)
firstNodeNumber=1
nodes = iron.Nodes()
region.NodesGet(nodes)
lastNodeNumber = nodes.numberOfNodes
firstNodeDomain = decomposition.NodeDomainGet(firstNodeNumber,1)
lastNodeDomain = decomposition.NodeDomainGet(lastNodeNumber,1)
if firstNodeDomain == computationalNodeNumber:
    boundaryConditions.SetNode(dependentField,iron.FieldVariableTypes.U,1,1,firstNodeNumber,1,iron.BoundaryConditionsTypes.FIXED,0.0)
if lastNodeDomain == computationalNodeNumber:
    boundaryConditions.SetNode(dependentField,iron.FieldVariableTypes.U,1,1,lastNodeNumber,1,iron.BoundaryConditionsTypes.FIXED,1.0)
solverEquations.BoundaryConditionsCreateFinish()

In [21]:
# Solve the problem
problem.Solve()

In [22]:
# Now we want to have the results of the run be stored for analysis and later use

# Export results
baseName = "laplace"
dataFormat = "PLAIN_TEXT"
fml = iron.FieldMLIO()
fml.OutputCreate(mesh, "", baseName, dataFormat)
fml.OutputAddFieldNoType(baseName+".geometric", dataFormat, geometricField,
    iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES)
fml.OutputAddFieldNoType(baseName+".phi", dataFormat, dependentField,
    iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES)
fml.OutputWrite("LaplaceExample.xml")
fml.Finalise()

In [23]:
# Let the library know that you are done with computations and the resources allocated for the problem
# can now be released
iron.Finalise()